In [0]:
%pip install httpx

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
# Set environmental variables for testing purposes 
# TODO: TO BE REMOVED!
import os
os.environ["STORAGE_ACCOUNT"] = r"adlsinframricdev"
os.environ["STORAGE_CONNECTOR"] = r"adlsinframricdev.dfs.core.windows.net"
os.environ["STORAGE_ACCESSOR"] = r"fs.azure.account.key.adlsinframricdev.blob.core.windows.net"
os.environ["STORAGE_KEY"] = r""
os.environ["BRONZE_CONTAINER"] = r"bronze"
os.environ["SILVER_CONTAINER"] = r"silver"
os.environ["GOLD_CONTAINER"] = r"gold"
os.environ["DATA_CATALOGUE_URL"] = r"http://davidsalac.eu/mcdatacatalogue/catalogue.json"
os.environ["HASH_SALT"] = r""
os.environ["CLIENT_ID_HASH_SUFFIX"] = r"_Hash"



In [0]:
import os

from pyspark.sql import SparkSession
from pyspark.sql.functions import sha2, lit, concat, date_trunc
import httpx

# === CONFIG SECTION ===
STORAGE_ACCOUNT: str = os.environ.get('STORAGE_ACCOUNT')
STORAGE_CONNECTOR: str = os.environ.get('STORAGE_CONNECTOR')
STORAGE_ACCESSOR: str = os.environ.get('STORAGE_ACCESSOR')
STORAGE_KEY: str = os.environ.get('STORAGE_KEY')
# a) Bronze Storage Container name:
BRONZE_CONTAINER: str = os.environ.get('BRONZE_CONTAINER')
# b) Silver Storage Container name:
SILVER_CONTAINER: str = os.environ.get('SILVER_CONTAINER')
# c) Gold Storage Container name
GOLD_CONTAINER: str = os.environ.get('GOLD_CONTAINER')
# d) Link to data catalogue
DATA_CATALOGUE_URL: str = os.environ.get('DATA_CATALOGUE_URL')
# e) Treatment for ClientID column:
CLIENT_ID_HASH_SUFFIX: str = os.environ.get('CLIENT_ID_HASH_SUFFIX')
HASH_SALT: str = os.environ.get('HASH_SALT')
# ~~~~~~~~~~~~~~~~~~~~~~

# === SET UP SPARK SESSION ===
#session = SparkSession.builder.getOrCreate()
#session.conf.set(STORAGE_ACCESSOR, STORAGE_KEY)
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# === READ DATA CATALOGUE ===
data_catalogue_connector = httpx.get(DATA_CATALOGUE_URL, timeout=5)
data_catalogue: dict = data_catalogue_connector.json()
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [0]:
# retreive ADF parameters
kv_client_id = dbutils.widgets.get('client_id')
kv_client_secret = dbutils.widgets.get('client_secret')

In [0]:
#service_credential = dbutils.secrets.get(scope="flowehr-secrets",key="<service-credential-key>")
client_id = dbutils.secrets.get(scope="flowehr-secrets", key= kv_client_id)#"flowehr-dbks-adls-app-id")
client_secret = dbutils.secrets.get(scope="flowehr-secrets", key= kv_client_secret) #"flowehr-dbks-adls-app-secret")

spark.conf.set("fs.azure.account.auth.type.adlsinframricdev.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.adlsinframricdev.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.adlsinframricdev.dfs.core.windows.net", client_id)
spark.conf.set("fs.azure.account.oauth2.client.secret.adlsinframricdev.dfs.core.windows.net", client_secret)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.adlsinframricdev.dfs.core.windows.net", "https://login.microsoftonline.com/<tenant-id>/oauth2/token")

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit

# Set up the SparkSession
spark = SparkSession.builder.appName("ReadParquetFilesFromBlobStorage").getOrCreate()

# Set up the Azure Blob Storage account details
account_name = ""
account_key = ""

In [0]:

# === FUNCTIONALITY BLOCK ===
def hash_client_id(data_frame, columns: list[str]):
    """Create salted hash of the ClientID column.
    Args:
        data_frame: Data frame with potential ClientID column
        columns (list[str]): List of columns with ClientID.
    Returns:
        Salted and hashed ClientID column inside a new Data Frame
    """
    hashed_client_id_df = data_frame
    for _column in columns:
        # Merge ClientID with HASH_SALT (salting procedure)
        hashed_client_id_df = hashed_client_id_df.withColumn(
            _column,
            concat(hashed_client_id_df[_column], lit(HASH_SALT))
        )
        # Compute hash value
        hashed_client_id_df = hashed_client_id_df.withColumn(
            "".join([_column, CLIENT_ID_HASH_SUFFIX]),
            sha2(
                hashed_client_id_df[_column].cast("Binary"), 256
            )
        )
        # Drop old columns
        hashed_client_id_df = hashed_client_id_df.drop(_column)

    return hashed_client_id_df


def round_datetime_columns(data_frame, columns: list[str], round_option: str):
    """Rounds datetime columns on required option.
    Args:
        data_frame: Input dataframe.
        columns (list[str]): List of columns which are rounded.
        round_option (str): Option for rounding
            (defined by PySpark, mainly "hour" and "month")
    Returns:
        New dataframe with rounded datetime columns.
    """
    datetime_rounded_df = data_frame
    for _column in columns:
        datetime_rounded_df = datetime_rounded_df.withColumn(
            _column,
            date_trunc(round_option, data_frame[_column])
        )
    return datetime_rounded_df
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~


# === MAIN APPLICATION LOOP ===
# Goes through all tables in the bronze data storage
for table_name, table in data_catalogue.items():
    # Get file name in bronze storage (TableName.txt - without dots)
    bronze_directory = "raw/"
    silver_directory = "transformed/"
    gold_directory = "curated/"

    file_name_bronze = f'{bronze_directory}{table_name.replace("dbo.", "")}.parquet'
    file_name_silver = f'{silver_directory}{table_name.replace("dbo.", "")}'
    file_name_gold = f'{gold_directory}{table_name.replace("dbo.", "")}'

    # Read the file into Spark data frame
    raw_table_df = spark.read.format("parquet").load(f"abfs://{BRONZE_CONTAINER}@{STORAGE_CONNECTOR}/{file_name_bronze}")

    # Hash ClientID column with salt
    hashed_client_id_table_df = hash_client_id(raw_table_df,
                                               table['client_id'])

    # Drop all identifiable columns
    if len(table['other_identifiable_columns']) != 0 :
        deidentified_table_df = hashed_client_id_table_df.drop(*table['other_identifiable_columns'])
    else:
        deidentified_table_df = hashed_client_id_table_df

    # Truncate all times to hours
    hours_truncated_df = round_datetime_columns(deidentified_table_df,
                                                table['date_time'],
                                                "hour")

    # Truncate all date of births to months
    day_truncated_df = round_datetime_columns(hours_truncated_df,
                                              table['date_of_birth'],
                                              "month")

    # Check if there is anything to be written (skips empty tables)
    if len(
        set(table['columns_descriptions'].keys()) - (
            set(table['other_identifiable_columns'])
        )
    ) == 0:
        continue

    # Write table into silver storage
    day_truncated_df.write.format('parquet').mode('overwrite').save(f"abfs://{SILVER_CONTAINER}@{STORAGE_CONNECTOR}/{file_name_silver}")

    # Drop columns with free text
    if len(table['free_text_columns']) != 0 :
        no_free_text_table_df = day_truncated_df.drop(*table['free_text_columns'])
    else:
        no_free_text_table_df = day_truncated_df

    # Check if there is anything to be written (skips empty tables)
    if len(
        set(table['columns_descriptions'].keys()) - (
            set(table['free_text_columns']) |
            set(table['other_identifiable_columns'])
        )
    ) == 0:
        continue

    # Write table into gold storage
    no_free_text_table_df.write.format('parquet').mode('overwrite').save(f"abfs://{GOLD_CONTAINER}@{STORAGE_CONNECTOR}/{file_name_gold}")


In [0]:
# For debugging and testing purposes (To Be Deleted)
#test = session.read.parquet(
#    f"wasbs://{GOLD_CONTAINER}@{STORAGE_CONNECTOR}/dboAmsAppointmentContact.txt"
#)
#test.display()

SequenceID,ReferralID,InitialContactFlag,ActualTime,ActualDuration,ArrivalTime,Outcome,CancellationDateTime,CancellationReason,CancellationBy,NonClientAppointmentFlag,FaceToFaceContact,StaffProfessionalGroup,ContactID,TimeStamps,AppointmentConsultationMedium,ProxyContact,SecondArrivalTime,BookedStatus,InterpreterStatus,InterpreterLanguage,TransportStatus,CancellationOnBehalfOf,CancellationDateTimestamp,IsFamilyMember,DischargeReport,InviteType,ePathwayID,PerinatalMhAss,IMCA,IMHA,ReasonableAdjMade,ClientID_Hash,RelatedClientID_Hash
2149,27706,true,2013-01-23T19:00:00.000+0000,23316,2027-04-27T20:00:00.000+0000,RSeAQB0dbVvWcAadEAoC,1982-10-24T19:00:00.000+0000,qzHeUBbOeWCd9KSnTZw3,RN1sW3JgD0iZFuBvEtoP,false,true,JsqIL3AkKgFTQtCP0cs5,13528,AAAAAAAAAAAAAAAAAAAAAAAME+I=,ROmssl4Rhb8AfoRrACRj,true,2007-09-05T05:41:36.000+0000,DnNR7tnzoaiX5yED97MM,s3jbVZBShRClBYvbEcei,E94U4gPbK9Ynn3lfWCUs,y5Kk1YNnWlHwunNvS23V,QcFsARp2VDSiYsnULVDy,2010-10-24T11:19:54.000+0000,false,31796,12064,21275,CpeIyg1Gv67JM06NBADj,goFDN5xH1TmjOcB39Xpa,fA1GEXPqYNv47QfWCbwq,YMK8AacIXl4BZ5UQACI2,455ac7b4b8007989a6e92e5477e83a776a0903100a8da52fe149df51ca107f15,e485e8039514654a7e0d5a5b77135aa42db799b3cd5ce6df268730ae20929802
6089,1821,true,2009-12-06T21:00:00.000+0000,19209,1977-04-22T19:00:00.000+0000,uxQf72UMuh5RuhO0riKx,2019-06-04T00:00:00.000+0000,KkGCsxQAtEYDlYlbCDph,Tu23Aj1KEaJxX5f4Gsw9,false,true,2CeEMj8FVh46oqo1n3cc,14252,AAAAAAAAAAAAAAAAAAAAAAAE0qY=,RfHOkyNm8EuFh8du1hbT,false,1971-02-18T17:57:31.000+0000,p92G38FXKZeNFYUH7lUK,J1ZhJVZBRce79mIBmLF9,cuSvZ9izVCc2jvutdIfV,IKKtf2pdLMq58EzySVr6,i2aU4oDRkYn0aluZxwBk,2001-06-11T02:43:43.000+0000,true,15732,24592,29684,1Rdki3GYLwlDJdYkd037,fZIsHEOGpquFyb1JMThc,ooADVLNP1ibUPGLPyd3C,OT6Cy4IwsTckKTvDsFta,40745ddb4dad00609b4f0d09243d80e12fea0195984c1cb56030c36ab1bfbaff,8816d5f8393c546218a8c178764bebc43894fa7fef65252a20346f348a9fa455
13086,18106,false,2013-11-27T05:00:00.000+0000,1924,1993-07-04T21:00:00.000+0000,hp4PL2C1P91L59cuoscL,2022-05-27T00:00:00.000+0000,0vM7bKOPocPagmyV3WMz,9QsZvVPXCyvIBvOJpfB8,true,true,NHP0letwBWRixofq6Wp5,15306,AAAAAAAAAAAAAAAAAAAAAAAKFks=,2Yr7sPhcSJbmhvTbkR3v,true,2016-07-11T11:43:32.000+0000,RJ0R7tyhSLDYgu2w5eVL,dTRxuv4jWL2n1uFbxLzh,NdtrodJESimlw4fvC6yU,8usHRMuC89ZvIlW6oZe7,0SuFOdjkHVtTso5SF9j9,1989-05-24T05:40:17.000+0000,false,15311,3876,15887,oyO5LRP1oPM72MKsFz3R,zYRsUbHpQDDqYBjBo2Fs,MSbZZgWehj3QJxM1vhrV,UV6YTHztA4atFmvpw43h,fb53d833c14655f691033be2a865423d348a117abd1a9f8cf076adf7dc812d66,76ba1feea437049ad0d2e8800fc28654e29a6f0e023f7df552fe680bd7ecabf6
15373,4053,true,1992-10-16T17:00:00.000+0000,13159,2017-04-20T22:00:00.000+0000,Gu09OrMdIoZyEnZcNf5L,2023-02-06T09:00:00.000+0000,ItoQv7UBI7DMqvUUGnH8,JDNL9h2bXM4Yz9DKTgZz,true,true,qOq9Nz9a11WkdblICpiC,31777,AAAAAAAAAAAAAAAAAAAAAAAIn8g=,v9m3FZvY5Z3fsDjf3cBt,false,1984-06-18T20:37:55.000+0000,01vNpdbHdGHCD7FsomEY,LMmsRQ7xn16pBc5gmNfp,dWmMBmjbSIIYwD8z8KwJ,rzLHCK9FEkuhPa4EhyrO,ZhuHpZICFhqgnz4fWJA3,1994-04-25T02:07:54.000+0000,true,11130,12202,21604,oZ4AAowtRXAMq8C9lEv8,D3HaTzgWRswzhZLU2Yw8,igfrjrIaStg98C31UikL,9xXVnbwZyYDfG8bPK8xO,f227726dd1037ea38a1cb1fca6a33a2db49c2259017278bea5c9e3e20ab5c00b,5105e44953a67d13546fde284f967de4ea0370b98e6d8f94844ff56393256d6a
16519,6423,false,1980-10-07T15:00:00.000+0000,9043,1983-08-24T11:00:00.000+0000,k2WQ8wMC8lDgUVYm9qwi,1988-01-01T01:00:00.000+0000,n0pKx8y6le6WuSjWSld7,Y06PDp9aRdh1RpaxRXR2,false,true,Ob2CYSWNNY9YecWCPoax,19379,AAAAAAAAAAAAAAAAAAAAAAAHVvM=,chhkJCvyL0MNJLGEgsuU,true,1975-08-19T15:01:45.000+0000,gQRWa7IxOj0Dv9tJO5jZ,yTbPCj4tuKJW8jexFiGj,MNnkX78zmtTVGHqzyNTq,1fKIEl9WKiyjlR08EVSS,hGWX920HkqKPbU3FQLBy,2017-08-22T20:07:54.000+0000,false,870,16201,13638,hbaV3Kz5oEyOKxVcw2Rq,ZpEuAObSkGVLoWxdqgVh,OoAUagoAtVuAmkkO0o9E,9BUJ93VRmixIIPPb7wnB,7bd43a6defd11ae248bef492abd34cab832735eb5e8be0d90b20ba8401e33c8d,9a32bf0798951be187967c3c448fbd1d2571fbcb13e8d1c6d89e18457423854f
18947,12156,true,2014-04-20T10:00:00.000+0000,8186,1979-02-13T07:00:00.000+0000,ea7f7Y2BNBHAZqbIdhvm,2019-07-18T10:00:00.000+0000,MZNsXBGp